In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [131]:
data = pd.read_csv('data/train_data.csv', error_bad_lines=False)

/var/folders/51/svyxncy90c19w1j2cby059bm0000gn/T/ipykernel_19821/2132394297.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('data/train_data.csv', error_bad_lines=False)
b'Skipping line 5856: expected 40 fields, saw 70\nSkipping line 7939: expected 40 fields, saw 49\n'


In [32]:
lot_f = 'LTFRONT'
lot_d = 'LTDEPTH'
bld_f = 'BLDFRONT'
bld_d = 'BLDDEPTH'

area_data = pd.DataFrame(data[(data[lot_f] > 0) & (data[lot_d] > 0) & (data[bld_f] > 0) & (data[bld_d] > 0)], columns=[lot_f, lot_d, bld_f, bld_d])

In [41]:
front_mean = (area_data[lot_f] - area_data[bld_f]).mean()
front_std = (area_data[lot_f] - area_data[bld_f]).std()
depth_mean = (area_data[lot_d] - area_data[bld_d]).mean()
depth_std = (area_data[lot_d] - area_data[bld_d]).std()


print(f'Front: mean {front_mean}\t std: {front_std}\nDepth: mean {depth_mean}\t std: {depth_std}')

Front: mean 11.060518034374244	 std: 45.07528268290128
Depth: mean 49.48995400629388	 std: 49.1858577008738


In [103]:
def get_mean_column_value_for_boro(data, column_name: str, boro: int, include_zeros: bool = False):
    column_vals = data[data['BORO'] == boro][column_name]
    if include_zeros:
        return np.mean(column_vals)
    else:
        return np.mean(column_vals[column_vals != 0])

In [134]:
def fill_missing_column_values_with_mean_for_boro(data, column_name: str):
    data_filled = data.copy()
    for i, row in data_filled.iterrows():
        data_filled.at[i, column_name] = get_mean_column_value_for_boro(data_filled, column_name, row['BORO'])
    return data_filled

In [130]:
def replace_lengths_with_areas(data):
    data_with_area = data.copy()
    lot_f = 'LTFRONT'
    lot_d = 'LTDEPTH'
    bld_f = 'BLDFRONT'
    bld_d = 'BLDDEPTH'
    data_with_area['LTAREA'] = data[lot_f] * data[lot_d]
    data_with_area['BLDAREA'] = data[bld_f] * data[bld_d]
    data_with_area['LTAREA'] = np.where(data_with_area['LTAREA'] == 0, data_with_area['BLDAREA'], data_with_area['LTAREA'])
    data_with_area['BLDAREA'] = np.where(data_with_area['BLDAREA'] == 0, data_with_area['LTAREA'], data_with_area['BLDAREA'])
    
    for len_col_name in [lot_f, lot_d, bld_f, bld_d]:
        del data_with_area[len_col_name]
    return data_with_area

In [57]:
data_with_area = data.copy()
data_with_area['LTAREA'] = data[lot_f] * data[lot_d]
data_with_area['BLDAREA'] = data[bld_f] * data[bld_d]

In [61]:
data_with_area['LTAREA'] = np.where(data_with_area['LTAREA'] == 0, data_with_area['BLDAREA'], data_with_area['LTAREA'])

In [132]:
data_with_area = replace_lengths_with_areas(data)

In [137]:
data_with_area_filled = fill_missing_column_values_with_mean_for_boro(data_with_area, 'LTAREA')
data_with_area_filled = fill_missing_column_values_with_mean_for_boro(data_with_area_filled, 'BLDAREA')

In [113]:
get_mean_column_value_for_boro(data_with_area, 'LTAREA', 1)

20278.494930875644

In [93]:
column_vals = data_with_area[data['BORO'] == 1]['LTAREA']

In [101]:
from sklearn import preprocessing
avland = data['AVLAND'].to_numpy()
std_scaler = preprocessing.StandardScaler()
std_scaler.fit(avland.reshape(-1, 1))
avland_scaled = std_scaler.transform(avland.reshape(-1, 1))
# plt.plot(avland_scaled)
avland_scaled

array([[ 0.01834275],
       [-0.05684651],
       [-0.06132275],
       ...,
       [-0.06345824],
       [-0.06243566],
       [-0.06008669]])

In [147]:
import src.Preprocessing as pp
import importlib
importlib.reload(pp)

preproc = pp.Preprocessing('data/train_data.csv')
X, y = preproc.run(0.7, ['2'], 1)

            FULLVAL
FULLVAL    1.000000
STORIES   -0.123290
AVLAND     0.658087
AVTOT      1.000000
EXLAND     0.521816
EXTOT      0.611863
AVLAND2    0.699934
AVTOT2     0.988892
EXLAND2    0.516609
EXTOT2     0.604629
Latitude   0.305759
Longitude  0.313090
LTAREA     0.317053
BLDAREA    0.217378


In [146]:
X

,STORIES,AVLAND,AVTOT,EXLAND,EXTOT,AVLAND2,AVTOT2,EXLAND2,EXTOT2,Latitude,Longitude,LTAREA,BLDAREA
0,0.441102,-0.173742,-0.312173,0.165863,-0.004853,-0.178955,-0.307608,0.198283,0.007340,-0.228062,-0.289753,-0.474110,-0.464399
1,0.577516,-0.211208,-0.352585,-0.210752,-0.229057,-0.222434,-0.354913,-0.214121,-0.230640,-0.245862,-0.126036,0.778021,0.879112
2,0.713929,-0.193859,-0.346498,0.095769,-0.062069,-0.202300,-0.341757,0.121527,-0.049821,-0.224739,-0.313780,-0.474110,-0.464399
3,0.168275,-0.218568,-0.142727,-0.210752,-0.229057,-0.230975,-0.155786,-0.214121,-0.230640,-0.104787,-0.090177,-0.104431,-0.067741
4,-0.445586,0.144921,0.362670,-0.210752,-0.229057,0.190856,0.435277,-0.214121,-0.230640,-0.071581,-0.124053,-0.474110,-0.464399
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,-0.650206,-0.258970,-0.420656,-0.210752,-0.229057,-0.277731,-0.423406,-0.214121,-0.230640,-1.939092,-2.653920,-0.474110,-0.464399
2585,-0.991240,-0.260757,-0.430938,-0.210752,-0.229057,-0.279935,-0.433945,-0.214121,-0.230640,-1.858626,-2.566661,-0.474110,-0.464399
2586,-0.650206,-0.245725,-0.405574,-0.210752,-0.229057,-0.262267,-0.408117,-0.214121,-0.230640,-1.939092,-2.653920,-0.474110,-0.464399
2587,-0.650206,-0.258676,-0.422063,-0.210752,-0.229057,-0.277521,-0.425083,-0.214121,-0.230640,-2.031541,-2.608039,-0.474110,-0.464399


In [128]:
data['AVLAND2'] = data['AVLAND2'].fillna(0)

0        118035.0
1         17301.0
2         11304.0
3         14081.0
4         10126.0
           ...   
11941         0.0
11942         0.0
11943         0.0
11944         0.0
11945         0.0
Name: AVLAND2, Length: 11946, dtype: float64